In [1]:
# ADMM adaptive lasso using ProximalOperators with line and golden section search
# Automatic tuning of learning rate and regularization parameter
# This code only only changes the ratio between test data and train data, and changes the input file
# The original source code from https://github.com/patwa67/AUTALASSO

using ProximalOperators
using DelimitedFiles
using Statistics
using LinearAlgebra

In [2]:
# Function for Golden section search to optimize lambda
function gss_opt(alam, blam, tolgss, Xtesthot, ytest,abscovinv,maxnorm)
  lama =alam*maxnorm # Lower lambda
  lamb =blam*maxnorm # Higher lambda
  gr = (sqrt(5.0) + 1.0) / 2.0 # Golden section ratio
  toladmm = 1e-4 # Convergence tolerance for ADMM
  fc = lasso_admm(Xtrainhot, ytrain, lama, zero(Xtrainhot[1,:]),zero(Xtrainhot[1,:]),f, abscovinv,toladmm)
  lossc= 0.5*norm(Xtesthot*fc[1].-ytest)^2 # Test error for initial lower lambda
  fd = lasso_admm(Xtrainhot, ytrain, lamb, zero(Xtrainhot[1,:]),zero(Xtrainhot[1,:]),f, abscovinv,toladmm)
  lossd= 0.5*norm(Xtesthot*fd[1].-ytest)^2 # Test error for initial higher lambda
  iter = 2
  meanlam = zero(1.0:100.0)
  #meanlam[iter] = (lama+lamb)/2
  meanloss = zero(1.0:100.0)
  #meanloss[1] = max(lossc,lossd)
  #meanloss[iter] = (lossc+lossd)/2
  lamc = lamb - (lamb - lama) / gr
  lamd = lama + (lamb - lama) / gr
  println("lossc =$lossc")
  println("lossd =$lossd")
  println("lambdaa =$lama")
  println("lambdab =$lamb")
  println("lambdac =$lamc")
  println("lambdad =$lamd")
  iter = 2
  nodrun = 0
  while abs(lamc - lamd)/((lamc + lamd)/2.0) > tolgss # Run GSS until convergence
    iter = iter+1
    if iter == 3
    fc = lasso_admm(Xtrainhot, ytrain, lamc, fc[1],fc[2],f, abscovinv,toladmm)
    lossc= 0.5*norm(Xtesthot*fc[1].-ytest)^2 # Test error for initial lower lambda
    fd = lasso_admm(Xtrainhot, ytrain, lamd, fd[1],fd[2],f, abscovinv,toladmm)
    lossd= 0.5*norm(Xtesthot*fd[1].-ytest)^2 # Test error for initial higher lambda
    else
    if nodrun==1
    fc = lasso_admm(Xtrainhot, ytrain, lamc, fc[1],fc[2],f, abscovinv,toladmm)
    lossc= 0.5*norm(Xtesthot*fc[1].-ytest)^2 # Test error for initial lower lambda
    else
    fd = lasso_admm(Xtrainhot, ytrain, lamd, fd[1],fd[2],f, abscovinv,toladmm)
    lossd= 0.5*norm(Xtesthot*fd[1].-ytest)^2 # Test error for initial higher lambda
    end
    end
    meanlam[iter] = (lamc+lamd)/2.0
    meanloss[iter] = (lossc+lossd)/2.0
    # Stop GSS if test MSE is increased two consecutive iterations
    if (meanloss[iter] > meanloss[iter-1])&&(meanloss[iter-1] > meanloss[iter-2])
      break
    end
    if lossc < lossd
      lamb = lamd
      fd=fc
      lossd=lossc
      nodrun=1
    else
      lama = lamc
      fc=fd
      lossc=lossd
      nodrun=0
    end
    lamc = lamb - (lamb - lama) / gr
    lamd = lama + (lamb - lama) / gr
    #println("lossc =$lossc")
    #println("lossd =$lossd")
    println("lambdac =$lamc")
    println("lambdad =$lamd")
  end
  # Final ADMM for optimized lambda
  lamopt = meanlam[iter-2]
  fmean1 = (fc[1]+fd[1])/2.0
  fmean2 = (fc[2]+fd[2])/2.0
  fopt = lasso_admm(Xtrainhot, ytrain, lamopt, fmean1,fmean2,f, abscovinv,toladmm)
  lossopt= 0.5*norm(Xtesthot*fopt[1].-ytest)^2
  acc = cor(Xtesthot*fopt[1],ytest)

  return lamopt,fopt,lossopt,acc
end

gss_opt (generic function with 1 method)

In [3]:
# Function for Proximal ADMM lasso with line search
function lasso_admm(Xtrainhot, ytrain, lam, theta, beta, f, abscovinv,tol; maxit=5000)
  u = zero(theta)
  grad = zero(theta)
  lamw = lam*abscovinv # Regularization parameter times weights
  g = NormL1(lamw) # Regularization function
  c = 0.5
  lr = 1.0
  loss(theta) = 0.5*norm(Xtrainhot*theta-ytrain)^2 # Loss function for line search
  for it = 1:maxit
    # Line search
    it % 8 == 1 && (grad = Xtrainhot'*(Xtrainhot*beta-ytrain))
    while  it % 20 == 2 && loss(theta) > (loss(beta) + grad'*(-beta) + (1.0/(2.0*lr))*norm(-beta)^2)
      lr = lr * c
      #println(lr)
    end
    gam = lr
    # ADMM perform f-update step
    prox!(beta, f, theta - u, gam)
    # ADMM perform g-update step
    prox!(theta, g, beta + u, gam)
    # Stopping criterion for ADMMM
    if norm(beta-theta, Inf) <= tol*(1+norm(u, Inf))
      break
    end
    # Dual update
    u .+= beta - theta
    #print(it)
  end
  return theta,beta,tol
end

lasso_admm (generic function with 1 method)

In [4]:
data_file_path = "/Users/dong/GWAS/AtPolyDB/Width_22.csv"

"/Users/dong/GWAS/AtPolyDB/Width_22.csv"

In [5]:
# Read Width_22.csv, and partition into train and test parts
X = readdlm(data_file_path,',')
ytot = (X[:,1].-mean(X[:,1])) # Center y to mean zero
ytrain = ytot[1:150]
Xtest= X[151:size(X)[1],2:size(X)[2]]
ytest = ytot[151:size(X)[1]]
Xtrain = X[1:150,2:size(X)[2]]

150×214051 Matrix{Float64}:
 0.0  0.0  0.0  2.0  2.0  0.0  0.0  2.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0     0.0  2.0  0.0  0.0  2.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0     0.0  0.0  0.0  0.0  0.0  2.0  0.0
 0.0  2.0  0.0  0.0  0.0  2.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  2.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0  …  2.0  2.0  0.0  0.0  2.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0     2.0  2.0  0.0  0.0  2.0  0.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0     2.0  2.0  0.0  0.0  2.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  2.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0     0.0  2.0  2.0  0.0  2.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  2.0
 0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  

In [6]:
# One hot encoding training data
Xtrain0 = copy(Xtrain)
Xtrain1 = copy(Xtrain)
Xtrain2 = copy(Xtrain)
Xtrain0[Xtrain0.==1] .= 2
Xtrain0[Xtrain0.==0] .= 1
Xtrain0[Xtrain0.==2] .= 0
Xtrain1[Xtrain1.==2] .= 0
Xtrain2[Xtrain2.==1] .= 0
Xtrain2[Xtrain2.==2] .= 1
Xtrainhot = hcat(Xtrain0,Xtrain1,Xtrain2)
# Set unimportant allocations to zero
Xtrain0 = 0
Xtrain1 = 0
Xtrain2 = 0
Xtrain = 0

0

In [7]:
# One hot encoding test data
Xtest0 = copy(Xtest)
Xtest1 = copy(Xtest)
Xtest2 = copy(Xtest)
Xtest0[Xtest0.==1] .= 2
Xtest0[Xtest0.==0] .= 1
Xtest0[Xtest0.==2] .= 0
Xtest1[Xtest1.==2] .= 0
Xtest2[Xtest2.==1] .= 0
Xtest2[Xtest2.==2] .= 1
Xtesthot = hcat(Xtest0,Xtest1,Xtest2)
# Set unimportant allocations to zero
Xtest0 = 0
Xtest1 = 0
Xtest2 = 0
Xtest = 0

0

In [13]:
# Factor for initial lower lambda
alam = 0.0001
# Factor for initial upper lambda
blam = 1.0
# Convergence factor for lambda in gss_opt
tolgss = 0.01
# Find lambda where all reg coeff are zero
maxnorm = norm(Xtrainhot'*ytrain, Inf)
# The least squares loss function
f = LeastSquares(Xtrainhot, ytrain)
# Inverse covariances to be used as weights in the adaptive lasso
abscovinv = 1.0./abs.(cov(Xtrainhot,ytrain))

642153×1 Matrix{Float64}:
   74.99999999999997
  256.89655172413757
   84.819734345351
   23.041237113402072
  120.81081081081071
   11.948676824378506
   54.05078597339782
  161.3718411552353
   31.128133704735372
   21.084905660377363
 1090.2439024390208
  519.7674418604649
   57.603092783505154
    ⋮
  111.19402985074618
   61.48555708390638
   22.117763483424042
   11.601349597716062
  451.51515151514883
   35.05882352941176
   46.46569646569648
   10.388101324657214
   30.763936682725422
   20.438957475994513
   16.604754829123312
   19.88434163701066

In [14]:
# Run AUTALASSO with timing
@time res = gss_opt(alam, blam, tolgss, Xtesthot, ytest,abscovinv,maxnorm)


lossc =48.86055178143158
lossd =49.39170204081634
lambdaa =0.006593999999999991
lambdab =65.93999999999991
lambdac =25.190914097953723
lambdad =40.75567990204619
lambdac =40.7556799020462
lambdad =50.37523419590744
lambdac =34.810468391814965
lambdad =40.7556799020462
lambdac =31.136125608184962
lambdad =34.81046839181496
lambdac =28.86525688158372
lambdad =31.136125608184962
lambdac =27.461782824554962
lambdad =28.865256881583722
lambdac =26.594388154982482
lambdad =27.461782824554962
lambdac =27.461782824554962
lambdad =27.997862212011242
lambdac =27.130467542438762
lambdad =27.461782824554962
134.929756 seconds (9.56 k allocations: 15.968 GiB, 2.37% gc time)


(27.02808548976872, ([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [6.611078702017026e-7, 4.7204683493285533e-7, 6.326004901743257e-7, 7.325009166534314e-7, 6.853675032286114e-7, 1.263060048117999e-7, 5.443274508242413e-7, 7.354995049485514e-7, 6.574592242769225e-7, 7.28967577894458e-7  …  1.9871509276482883e-8, -6.854315619814612e-8, 2.5216649999643645e-8, 7.230285554401031e-9, 3.604375531366706e-7, -1.8133799440736846e-8, 1.355402846207242e-7, 3.3307180988701455e-7, 6.717901894623055e-8, 1.955891842386973e-7], 0.0001), 49.326934589577675, 0.1306662342205459)

In [15]:
output_dir = "/Users/dong/GWAS/Autalasso/" 


"/Users/dong/GWAS/Autalasso/"

In [16]:
# Save regression coefficients, lambda, MSE and ACC to text files
writedlm(joinpath(output_dir, "outbeta_Width22.txt"), res[2][1])
writedlm(joinpath(output_dir, "outlambda_Width22.txt"), res[1])
writedlm(joinpath(output_dir, "outMSE_Width22.txt"), res[3]/length(ytest)*2)
writedlm(joinpath(output_dir, "outACC_Width22.txt"), res[4])

In [17]:
# Predict observations for test data
ytesthat = Xtesthot*res[2][1]

25-element Vector{Float64}:
  0.013263438840265524
 -0.011337293313052136
 -0.0040583425057655505
  0.010429743413118775
  0.01739106713890421
  0.01739106713890421
  0.01739106713890421
 -0.014170988740198886
 -0.007209665014413448
  0.010429743413118775
 -0.004375969587266699
  0.01739106713890421
 -0.004375969587266699
  0.008863856703740732
 -0.007209665014413448
 -0.007209665014413448
 -0.007209665014413448
 -0.014170988740198886
 -0.011337293313052136
  0.013263438840265524
 -0.014170988740198886
  0.01739106713890421
 -0.011337293313052136
 -0.004375969587266699
 -0.011337293313052136